In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
import os
import warnings
from scipy.sparse import vstack
from scipy.sparse import save_npz
from scipy.sparse import load_npz
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import csr_matrix
import spacy
import re
# Load the spaCy model for English. This needs to be installed via spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

warnings.filterwarnings("ignore")
from collections import Counter

In [60]:
from tqdm.notebook import tqdm
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

word_threshold = 20

# Preprocessing function to be applied to the entire DataFrame
def preprocess_text(df):
    # Normalize text: replace dashes with spaces, remove non-alphabetic characters except spaces
    df['utterance'] = df['utterance'].str.replace('-', ' ').str.lower()
    df['utterance'] = df['utterance'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))
    return df

# Function to tokenize and lemmatize with word frequency filtering
def tokenize_and_lemmatize(text, word_frequencies):
    words = text.split()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in words]
    filtered_tokens = [word if word_frequencies.get(word, 0) >= word_threshold else 'xxxxx' for word in lemmatized_tokens]
    return ' '.join(filtered_tokens)

def preprocess_batch(batch, word_frequencies):
    # Apply tokenization and lemmatization with word filtering
    batch['utterance'] = batch['utterance'].apply(lambda x: tokenize_and_lemmatize(x, word_frequencies))
    batch['is_question'] = batch['utterance'].str.contains(r'\?').astype(int)
    return batch

# Main preprocessing function
def preprocess_dataframe(dataframe, batch_size=1000):
    # Apply initial preprocessing
    dataframe = preprocess_text(dataframe)
    
    # Calculate word frequencies
    all_words = ' '.join(dataframe['utterance']).split()
    word_frequencies = Counter(all_words)
    
    # Initialize progress bar
    progress = tqdm(total=len(dataframe), desc="Processing batches")
    
    processed_batches = []
    for start_row in range(0, len(dataframe), batch_size):
        end_row = start_row + batch_size
        batch = dataframe.iloc[start_row:end_row]
        processed_batch = preprocess_batch(batch, word_frequencies)
        processed_batches.append(processed_batch)
        
        # Update progress
        progress.update(len(batch))
        
    progress.close()  # Ensure the progress bar is closed after processing
    
    return pd.concat(processed_batches)

# Load the DataFrame
df = pd.read_csv('archive/utterances-2sp.csv')

# Process the DataFrame
df_processed = preprocess_dataframe(df)

# Save the processed DataFrame
df_processed.to_csv('archive/processed_utterances-2sp.csv', index=False)


Processing batches:   0%|          | 0/1240112 [00:00<?, ?it/s]

In [63]:
# Split each utterance into a list of words and explode the DataFrame to get a row per word
words_series = df_processed['utterance'].str.split().explode()

# Convert the series to a set to get unique words
unique_words = set(words_series)

# get word frequencies
word_frequencies = words_series.value_counts().to_dict()

In [64]:
print(f'Number of unique words: {len(unique_words)}')

# sort the words by frequency and display the most common words
sorted_word_frequencies = sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)
sorted_word_frequencies[:10]

Number of unique words: 17549


[('the', 943818),
 ('and', 544649),
 ('to', 525141),
 ('a', 524645),
 ('of', 488369),
 ('xxxxx', 444125),
 ('that', 392136),
 ('in', 353176),
 ('it', 286517),
 ('you', 275173)]

# B

In [67]:
episodes = df_processed['episode'].unique()

# Split the filtered episodes into training and testing sets
train_episodes = np.random.choice(episodes, int(0.8 * len(episodes)), replace=False)
train = df_processed[df_processed['episode'].isin(train_episodes)]
test = df_processed[~df_processed['episode'].isin(train_episodes)]

train_host = train[train['is_host'] == True]
train_guest = train[train['is_host'] == False]

def get_transition_matrix_efficient(df):
    # Split all utterances into words and flatten the list
    all_words = df['utterance'].str.split().explode()
    unique_words = all_words.unique()
    d = len(unique_words)
    
    # Map each unique word to an index
    word_to_index = {word: index for index, word in enumerate(unique_words)}
    
    # Initialize the transition counts matrix
    transition_counts = np.zeros((d, d), dtype=int)
    
    # Iterate over each utterance
    for utterance in df['utterance']:
        words = utterance.split()
        for i in range(1, len(words)):
            word1 = words[i - 1]
            word2 = words[i]
            if word1 in word_to_index and word2 in word_to_index:  # Check if both words are in the index
                index1 = word_to_index[word1]
                index2 = word_to_index[word2]
                transition_counts[index1, index2] += 1
                
    # Normalize the transition counts to get probabilities
    # To avoid division by zero, add a small number to the denominator
    transition_probabilities = transition_counts / (transition_counts.sum(axis=1)[:, np.newaxis] + 1e-10)
    
    return transition_probabilities, word_to_index

# Apply the efficient function to get the transition matrices
transition_matrix_host, word_to_index_host = get_transition_matrix_efficient(train_host)
transition_matrix_guest, word_to_index_guest = get_transition_matrix_efficient(train_guest)

In [69]:
transition_matrix_host.shape, transition_matrix_guest.shape

((17498, 17498), (17402, 17402))

# A

In [75]:
def tensor_trouble(df):
    # Unique words and mapping
    all_words = df['utterance'].str.split().explode()
    word_to_index = {word: index for index, word in enumerate(all_words)}
    unique_words = all_words.unique()
    d = len(unique_words)
    
    # Map each unique word to an index
    word_to_index = {word: index for index, word in enumerate(unique_words)}
    
    # Initialize the tensor: 2x2xd
    tensor = np.zeros((2, 2, d), dtype=int)
    
    # Group by episodes and concatenate text with speaker roles
    for episode, group in df.groupby('episode'):
        # Flatten all text into one string per episode
        text = " ".join(group['utterance'])
        words = text.split()

        # Replicate 'is_host' values for each word in the utterance
        roles = np.concatenate(group.apply(lambda row: [row['is_host']] * len(row['utterance'].split()), axis=1).values)
        
        for i in range(2, len(words)):
            # Get the current and next words
            word = words[i - 2]
            next_role = roles[i - 1]
            following_role = roles[i]

            # Get the current and next indices
            current_index = word_to_index[word]
            current_role_index = 0 if next_role else 1
            next_role_index = 0 if following_role else 1


            
            # Skip if any word is not in the index (unlikely given preprocessing, but safe practice)
            if current_index == -1:
                continue
            
            # Update the tensor based on speaker transitions
            tensor[current_role_index, next_role_index, current_index] += 1

    return tensor, word_to_index


tensor, word_to_index = tensor_trouble(df)

In [77]:
tensor.shape

(2, 2, 97570)